In [1]:
import pandas as pd
import sqlite3 as sql
import json
import os
import time
import datetime
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

conn = sql.connect('Tweets5.db')

In [58]:
assframe = pd.read_sql_query('SELECT in_reply_to_status_id, created_at, id, text, user_id FROM tweets',conn)

In [4]:
assframe['created_at'] = pd.to_datetime(assframe['created_at'])

In [62]:
assframe.shape[0]

3845544

In [11]:
def get_time(tweet):
    creatime = tweet['created_at']
    time_change = datetime.timedelta(minutes=28)
    endtime = creatime + time_change
    return [creatime, endtime]

def findtails(frame):
    tails = []
    possibleframe = frame[frame['in_reply_to_status_id'].notnull()]
    possibleframe = possibleframe.sort_values('created_at',ascending=False)
    for index, i in possibleframe.iterrows():
        creatime = i['created_at']
        time_change = datetime.timedelta(minutes=28)
        endtime = creatime + time_change
        replyframe = possibleframe[(possibleframe['created_at'] < endtime) & (possibleframe['created_at'] > creatime)]
        if replyframe[replyframe['in_reply_to_status_id'] == i['id']].shape[0] == 0:
            tails.append(i)
    return pd.DataFrame(tails)

In [12]:
def findparent(i,frame):
    creatime = i['created_at']
    time_change = datetime.timedelta(minutes=28)
    endtime = creatime - time_change
    timeframe = frame[(frame['created_at'] > endtime) & (frame['created_at'] < creatime)]
    masked_frame = timeframe[timeframe['id'] == i['in_reply_to_status_id']]
    if masked_frame.shape[0] != 0:
        parent = masked_frame.reset_index().loc[0]
        if parent['in_reply_to_status_id'] is None:
            print('succes')
        return parent
    
def getConvLoop(i,frame):
    global conversations
    top = (findparent(i[-1],frame))
    if top is None:
        return None
    if pd.isnull(top['in_reply_to_status_id']):
        conv = i
        conv.append(top)
        conversations.append(conv)
    else:
        conv = i
        conv.append(top)
        getConvLoop(conv,frame)

def retrieveConv(convlist):
    return pd.DataFrame(convlist).sort_values(by=['created_at'])

In [13]:
def getconversations(frame):
    global conversations
    conversations = list()
    iteration = 0
    tails = findtails(frame)
    for index, i in tails.iterrows():
        attempt = getConvLoop([i],frame)
        iteration += 1
        if iteration%1000 == 0:
            print(index)
    returnthing = conversations
    return conversations

In [14]:
asssframe = assframe
massivelist = list()
j = 0
while (j + 25000) < asssframe.shape[0]:
    s = j + 25000
    pepe = (getconversations(asssframe[j:s]))
    for i in pepe:
        massivelist.append(i)
    j = s
pepe = (getconversations(asssframe[j:asssframe.shape[0]]))
for i in pepe:
    massivelist.append(i)

6991
4284
881


In [36]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
save_object(massivelist,'conversations.pkl')